In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).




```
function ClickConnect() {
    var buttons = document.querySelectorAll("colab-dialog.yes-no-dialog paper-button#cancel"); 
    buttons.forEach(function(btn) { 
        btn.click(); 
    }); 
    console.log("1분마다 자동 재연결"); 
    document.querySelector("colab-toolbar-button#connect").click(); 
} 
setInterval(ClickConnect,1000*60);
```


In [ ]:
%matplotlib inline

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import pandas as pd

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cpu")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 데이터 파일 로딩


In [ ]:
# SOS - 문장의 시작
# EOS - 문장의 끝
SOS_token = 0
EOS_token = 1

# 충분히 길게 -> 합니다 문제 해결?
MAX_LENGTH = 512

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # SOS 와 EOS 포함

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

데이터 준비를 위한 전체 과정:

-  데이터 파일을 읽습니다.
-  영어-한국어 쌍을 생성합니다.
-  쌍을 이룬 문장들로 단어 리스트를 생성합니다.




In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # 파일을 읽고 문장만 분리

    # MS + Watcom 에러 메세지 데이터
    '''
    ms = pd.read_csv('ms_watcom.csv')
    eng_kor = pd.read_csv('eng_kor_news.csv')
    df1 = ms[['Q', 'A']]
    df2 = eng_kor[['영어', '한국어']]
    df2.columns = ['Q', 'A']
    df = pd.concat([df1, df2])
    '''
    data = pd.read_csv('/content/gdrive/MyDrive/different_data.csv')
    df = data[['Q', 'A']]

    # 모든 줄을 쌍으로 분리
    pairs = df.values.tolist()

    # Lang 인스턴스 생성
    # pairs = [list(reversed(p)) for p in pairs]
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return df, input_lang, output_lang, pairs

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    df, input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")

    try:
        for pair in pairs:
            input_lang.addSentence(pair[0])
            output_lang.addSentence(pair[1])
    except AttributeError:
        print(pair)

    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return df, input_lang, output_lang, pairs

df, input_lang, output_lang, pairs = prepareData('eng', 'kor', False)
print(df.head())

Reading lines...
Read 11730 sentence pairs
Counting words...
Counted words:
eng 18619
kor 38513
                                                   Q  \
0  The initialization unit 440 may include a thir...   
1  It will be possible to transform the startup e...   
2  1A shows a V2V operation, (b) a V2I operation,...   
3  The other person registration database 370 may...   
4  Meanwhile, when the selection of the target ob...   

                                                   A  
0  상기 초기화부(440)는 제 3 트랜지스터(T3) 및 제 4 트랜지스터(T4)를 포...  
1                         창업생태계를 민간 주도로 전환할 수 있게 된다.  
2  도 1의 (a)는 V2V 동작, (b)는 V2I 동작, (c)는 V2P 동작을 나타낸다.  
3  타인 등록 데이터베이스(370)는 한 명 또는 복수 명의 타인에 대한 등록 데이터를...  
4  한편, 제어부(130)는 대상객체(510)의 선택이 해제되면 선택속성의 복사를 해제...  


# Seq2Seq 모델


# 인코더

인코더는 입력 문장의 모든 단어에 대해 어떤 값을 출력하는 RNN입니다.





In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 디코더

디코더는 인코더 출력 벡터를 받아서 번역을 생성하기 위한 단어 시퀀스를
출력합니다.




In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Attention 디코더

문맥 벡터만 인코더와 디코더 사이로 전달 된다면, 단일 벡터가 전체 문장을
인코딩 해야하는 부담을 가지게 됩니다.

Attention은 디코더 네트워크가 자기 출력의 모든 단계에서 인코더 출력의
다른 부분에 "집중" 할 수 있게 합니다. 첫째 *Attention 가중치* 의 세트를
계산합니다. 이것은 가중치 조합을 만들기 위해서 인코더 출력 벡터와
곱해집니다. 그 결과(코드에서 ``attn_applied``)는 입력 시퀀스의
특정 부분에 관한 정보를 포함해야하고 따라서 디코더가 알맞은 출력
단어를 선택하는 것을 도와줍니다.

어텐션 가중치 계산은 디코더의 입력 및 은닉 상태를 입력으로
사용하는 다른 feed-forwad 계층인 ``attn`` 으로 수행됩니다.

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# 학습




## 학습 데이터 준비

학습을 위해서, 각 쌍마다 입력 Tensor(입력 문장의 단어 주소)와
목표 Tensor(목표 문장의 단어 주소)가 필요합니다. 이 벡터들을
생성하는 동안 두 시퀀스에 EOS 토큰을 추가 합니다.

In [ ]:
def indexesFromSentence(lang, sentence):
    ret = []
    for word in sentence.split(' '):
        try:
            ret.append(lang.word2index[word])
        except:
            pass

    return ret
    # return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

## 모델 학습



In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return target_length / loss.item()
    # return loss.item() / target_length

현재 시간과 진행률%을 고려해 경과된 시간과 남은 예상
시간을 출력하는 헬퍼 함수




In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

전체 학습 과정

-  타이머 시작
-  optimizers와 criterion 초기화
-  학습 쌍의 세트 생성
-  도식화를 위한 빈 손실 배열 시작




In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, save_every=1000, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            
            print("Saving model...")
            torch.save(encoder.state_dict(), "/content/gdrive/MyDrive/seq2seq_encoder.pt")
            torch.save(decoder.state_dict(), "/content/gdrive/MyDrive/seq2seq_decoder.pt")
            

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0


    showPlot(plot_losses)

## 결과 도식화


In [ ]:
import matplotlib.pyplot as plt
# plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

# 평가




In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
# 학습 데이터 셋에 있는 랜덤 문장으로 평가
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('>', pair[0])
        print('=', pair[1])
        print('<', output_sentence)
        print('')

def evaluateSentence(encoder, decoder, sentence):
    print('>', sentence)
    output_words, attentions = evaluate(encoder, decoder, sentence)
    output_sentence = ' '.join(output_words)
    print('<', output_sentence)
    print('')

# 학습

In [ ]:
import os

ENCODER_PATH = "/content/gdrive/MyDrive/seq2seq_encoder.pt"
DECODER_PATH = "/content/gdrive/MyDrive/seq2seq_decoder.pt"

hidden_size = 256

encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

# 모델 불러오기
if os.path.isfile(ENCODER_PATH) and os.path.isfile(DECODER_PATH):
    print("Load model...")
    encoder1.load_state_dict(torch.load(ENCODER_PATH))
    attn_decoder1.load_state_dict(torch.load(DECODER_PATH))

# trainIters(encoder1, attn_decoder1, 30000, print_every=5000, save_every=10000)
# trainIters(encoder1, attn_decoder1, 60000, print_every=5000, save_every=10000)
# trainIters(encoder1, attn_decoder1, 100000, print_every=5000, save_every=10000)
# trainIters(encoder1, attn_decoder1, 100, print_every=10)

Load model...


In [ ]:
def evaluateByGCC(encoder, decoder, n=10):
    df = pd.read_csv('GCC errors.csv')
    for item in df.itertuples():
      evaluateSentence(encoder, decoder, item[1])
      # print(item[1])

evaluateByGCC(encoder1, attn_decoder1)

> `##' at end of macro definition
< 따라서, 원격 경우 있는 사용자가 있는 있는 있다. <EOS>

> `##' at start of macro definition
< 본 실시예에서, 있어서, 시스템은 중 하나인 따라 특정 시간 있다. <EOS>

> `#elif' after `#else'
< 이는 경우 주 주 대한 않는 있다. 있다. <EOS>

> #error XXX
< 이러한 경우 경우 대한 대한 있는 있다. 있다. <EOS>

> `#XXX' not within a conditional
< 이러한 경우 가장 않은 않은 않은 않은 것으로 위해 것이 바람직하다. 있다. <EOS>

> `#include' expects "FILENAME" or <FILENAME>
< 또는 또는 또는 또는 또는 또는 또는 또는 또는 유사한 있다. <EOS>

> `#' operator is not followed by a macro argument name
< 이를 경우 상기 대한 의해 의해 의해 <EOS>

> ANSI C requires a named argument before `...'
< 디지털 관련 또한 또한 대한 관련 것으로 가지 것으로 것으로 나타났다. <EOS>

> arithmetic on pointer to an incomplete type
< 디지털 지연 따르면, 주행 기능을 따라 달라질 수 있다. <EOS>

> attempt to take address of bit-field structure member `XXX'
< 상기 광 내지 내지 도 내지 사이에 사이에 하나를 있다. <EOS>

> badly punctuated parameter list in `#define'
< 이러한 실시예에서, 수신 수신 기초하여 수신 내에 내에 내에 의해 <EOS>

> break statement not within loop or switch
< 이 경우 또는 또는 또는 않는 때 또는 또는 것으로 <EOS>

> called object

In [ ]:
# 다른 컴파일러의 오류 메시지로 테스트

def evaluateByMars(encoder, decoder, n=10):
    df = pd.read_csv('Digital Mars Compile errors.csv')
    for item in df.itertuples():
      evaluateSentence(encoder, decoder, item[2])
      # print(item[1])

evaluateByMars(encoder1, attn_decoder1)

> 'identifier' is a member of 'identifier' and 'identifier'
< 그 경우 그 중 하나인 및 및 및 있다. 수 있다. <EOS>

> 'identifier' is a pure virtual function
< 가상 가상 가상 가상 가상 가상 가상 가상 <EOS>

> 'identifier' is already defined
< 이러한 경우 주 경우 작은 한 있다. <EOS>

> 'identifier' is a virtual base class of 'identifier'
< 가상 가상 가상 위해 가상 위해 가상 가상 가상 위해 가상 <EOS>

> 'identifier' is far
< 이에 따라, 경우 경우 가장 많이 것으로 있다. <EOS>

> 'identifier' is not a class template
< 가장 가장 가장 가장 가장 가장 가장 것으로 결정할 수 있다. <EOS>

> 'identifier' is not a constructor
< 이러한 결과 경우 않은 않은 않은 않은 것으로 <EOS>

> 'identifier' is not a correct struct
< 이러한 이러한 대한 대한 대한 의해 것으로 <EOS>

> 'identifier' is not a member of enum 'identifier'
< 가장 가장 가장 가장 한 한 있다. <EOS>

> 'identifier' is not a member of struct 'identifier'
< 가장 가장 가장 가장 가장 한 있는 것이 있다. <EOS>

> 'identifier' is not a member of forward referenced struct 'identifier'
< 가장 가장 가장 가장 가장 가장 많이 것으로 것으로 나타났다. <EOS>

> 'identifier' is not a struct or a class
< 이를 또는 또는 또는 또는 또는 또는 또는 또는 있다. <EOS>

> 'identifie

In [ ]:
# 학습데이터에서 랜덤하게 테스트

evaluateRandomly(encoder1, attn_decoder1, n=3)